# Install and Import 

In [1]:
# !pip install tensorflow==2.15.0
# !pip install gym
# !pip install keras
# !pip install keras-rl2
# %pip install scikit-learn

In [2]:
# %pip install ipykernel
# %pip install --upgrade nbformat
# %pip install stable-baselines3[extra]
# %pip install gymnasium

In [3]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [4]:
import numpy as np
import pandas as pd
import random
import logging
import math
from model_config import Path
import os
import torch

In [5]:
import gymnasium as gym
from gym import Env
from gymnasium import spaces
from gymnasium.utils import seeding

In [6]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import SubprocVecEnv

In [7]:
import decimal
decimal.getcontext().prec = 28  # Increase precision

In [8]:
import plotly.express as px
import plotly.graph_objects as go

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Используем GPU
    print("CUDA доступна. Работаем на GPU.")
else:
    device = torch.device("cpu")   # Используем CPU
    print("CUDA не доступна. Работаем на CPU.")


CUDA доступна. Работаем на GPU.


# Open and Preprocessing Data

- Изменили window_length тепер оно работает и выставили его на 96
- Кроме того, модели подают нормализованную Close, а также RSI теперь без нормализации
- нормализация StandartScaler

### Settings

In [10]:
window_length = 48
nb_steps = 80000

model_num = 7
data_num = 3

seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed= seed)

### Open Data

In [11]:
# Загрузка данных
df = pd.read_csv(Path["dataset"](model_num, data_num))

In [12]:
zero_close_prices = df[df['Close_orig'] == 0]
print(f"Number of zero 'Close' prices after scaling: {len(zero_close_prices)}")

Number of zero 'Close' prices after scaling: 0


In [13]:
unique_values = df['Asset_ID_encoded'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25], dtype=int64)

In [14]:
value_counts = df['Asset_ID_encoded'].value_counts()
value_counts

Asset_ID_encoded
1     4388
23    4388
18    4388
0     4387
5     4387
16    4384
15    4384
6     4379
25    4372
19    4371
4     4367
14    4342
13    4334
20    4314
3     4313
8     4309
22    4219
9     4207
21    4093
24    3602
11    3589
10    3582
17    3120
7     2957
2     2931
12    1847
Name: count, dtype: int64

In [15]:
# Разделение данных временного ряда на тренировочную и тестовую выборки.
def train_test_split_time_series(df, train_size=0.8):
    split_index = int(len(df) * train_size)
    train_df = df.iloc[:split_index].reset_index(drop=True)
    test_df = df.iloc[split_index:].reset_index(drop=True)
    return train_df, test_df

In [16]:
# Выполнение разделения
train_df, test_df = train_test_split_time_series(df, train_size=0.8)
print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

Training data shape: (83163, 26)
Testing data shape: (20791, 26)


# Game Rule

In [17]:
class TradingEnv(gym.Env):
    """
    Environment for training an agent to trade on the exchange using a continuous action space.
    """
    metadata = {'render.modes': ['human']}

    def __init__(self, df):
        super(TradingEnv, self).__init__()

        # Save data and initialize parameters
        self.df = df.reset_index(drop=True)
        self.total_steps = len(self.df) - 1
        self.window_length = window_length

        # Find indices where a new asset starts
        self.asset_start_indices = self._find_asset_start_indices()
        print(self.asset_start_indices)

        # Define action space: Continuous action between -1 and 1
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)

        # Define observation space
        num_features = len(self.df.columns) - 1 # Вычли Close_orig
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.window_length, num_features), dtype=np.float32)

        # Initialize trading parameters
        self.fee_cost = 0.001
        self.initial_balance = 1000  # Starting balance
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.net_worth

        self.reward = 0
        self.current_step = self.window_length
        self.current_price = 0
        self.shares_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        self.now_token = (self.df.loc[self.current_step, 'Asset_ID_encoded'] - 1)
        self.prev_token = self.now_token

        self.hist = {
            "current_step": [],
            'balance': [],
            'net_worth': [],
            'shares_held': [],
            "token": [],
            "current_price": [],
            "reward": [],
            "action": [],
            'total_shares_sold': [],
            'total_sales_value': [],
        }

        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)

        if self.logger.hasHandlers():
            self.logger.handlers.clear()

        log_file = Path["train_log"](model_num, data_num + 1)
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.INFO)

        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        self.logger.addHandler(file_handler)
        logging.getLogger().handlers = []


    def _find_asset_start_indices(self):
        """
        Find the indices in the DataFrame where a new asset starts.
        """
        asset_ids = self.df['Asset_ID_encoded']
        start_indices = {0:0}
        for i in range(1, len(asset_ids)):
            if asset_ids[i] != asset_ids[i - 1]:
                start_indices[asset_ids[i]] = i

        return start_indices


    def reset(self, seed = seed, options=None, reset_hist=False):
        super().reset(seed= seed)
        self.logger.info("Environment reset")

        # Existing reset logic
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.net_worth
        self.shares_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Move to the next asset's starting index
        self.now_token += 1
        if self.now_token not in self.asset_start_indices:
            self.now_token = list(self.asset_start_indices.keys())[0]  # Loop back to the first asset

        self.current_step = self.asset_start_indices[self.now_token] + self.window_length
        self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
        self.prev_token = self.now_token

        self.logger.info(f"Starting new episode with token {self.now_token} at step {self.current_step}")

        # Reset hist only if reset_hist is True
        if reset_hist:
            self.hist = {
                "current_step": [],
                'balance': [],
                'net_worth': [],
                'shares_held': [],
                "token": [],
                "current_price": [],
                "reward": [],
                "action": [],
                'total_shares_sold': [],
                'total_sales_value': [],
            }

        observation = self._next_observation()
        info = {}
        return observation, info


    def _next_observation(self):
        frame = self.df.drop(columns = ['Close_orig']).loc[self.current_step - self.window_length + 1:self.current_step]
        # obs = np.concatenate([
        #     frame.values,
        #     [self.shares_held],
        #     [self.balance],
        #     [self.net_worth]
        # ])
        obs = frame.values
        return obs.astype(np.float32)


    def step(self, action):
        self.reward = 0
        if isinstance(action, (list, np.ndarray)):
            action = action[0]

        self.logger.info(f"Step: {self.current_step}, Action taken: {action}")
        self._take_action(action)

        self.prev_token = self.now_token

        terminated = False
        truncated = False

        self.current_step += 1  # Move to the next time step

        if self.current_step >= self.total_steps:
            terminated = True
        else:
            self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
            if self.now_token != self.prev_token:
                self.logger.info(f"Token change at step {self.current_step}: {self.prev_token} -> {self.now_token}")
                # self._sell_all_tokens()
                self.now_token -=1
                terminated = True

        if self.prev_net_worth != 0:
            net_worth_change = self.net_worth - self.prev_net_worth
            percent_change = (net_worth_change) / self.prev_net_worth * 100

            if percent_change > 0:
                self.reward += percent_change
            else:
                self.reward -= percent_change * 2  

            if self.net_worth < self.initial_balance * 0.5:
                self.reward += -50  
                self.logger.info("Net worth dropped below 50% of initial balance.")
                terminated = True
        else:
            self.logger.info("prev_net_worth == 0")

        self.prev_net_worth = self.net_worth

        obs = self._next_observation()
        info = {}

        self.logger.info(f"Net worth: {self.net_worth}, Balance: {self.balance}, Reward: {self.reward}")

        self.hist["current_step"].append(self.current_step)
        self.hist["balance"].append(self.balance)
        self.hist["net_worth"].append(self.net_worth)
        self.hist["shares_held"].append(self.shares_held)
        self.hist["token"].append(self.now_token)
        self.hist["current_price"].append(self.current_price)
        self.hist["reward"].append(self.reward)
        self.hist["action"].append(action)
        self.hist["total_shares_sold"].append(self.total_shares_sold)
        self.hist["total_sales_value"].append(self.total_sales_value)

        return obs, self.reward, terminated, truncated, info


    def _sell_all_tokens(self):
        """
        Sell all tokens held at the current price.
        """
        self.current_price = self.df.loc[self.current_step - 1, 'Close_orig']
        shares_to_sell = self.shares_held
        if shares_to_sell > 0:
            # Determine total sale amount
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost  # Corrected: subtract transaction cost

            self.balance += total_sale
            self.shares_held = 0
            self.total_shares_sold += shares_to_sell  # Corrected: use shares_to_sell
            self.total_sales_value += total_sale

            # Update net worth
            self.net_worth = self.balance

            self.logger.info(f"Sold all shares of token {self.prev_token} at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")
        else:
            self.logger.info("No shares to sell.")


    def _take_action(self, action):
        """
        Apply the continuous action to the current state.
        """
        self.current_price = self.df.loc[self.current_step, 'Close_orig']

        action = float(np.clip(action, -1, 1))

        if action < 0:
            proportion = -action  # Convert to positive
            shares_to_sell = int(self.shares_held * proportion)
            self._sell(shares_to_sell)

        elif action > 0:
            proportion = action
            self._buy(proportion)
        else:
            # Hold
            self.reward += -1
            pass  

        self.net_worth = self.balance + self.shares_held * self.current_price


    def _buy(self, proportion):
        amount_to_spend = self.balance * proportion

        shares_to_buy = int(amount_to_spend / (self.current_price * (1 + self.fee_cost)))

        if shares_to_buy > 0:
            total_cost = shares_to_buy * self.current_price
            transaction_cost = total_cost * self.fee_cost
            total_cost += transaction_cost

            self.balance -= total_cost
            self.shares_held += shares_to_buy

            self.logger.info(f"Bought {shares_to_buy} shares at price {self.current_price}")
            self.logger.info(f"Total cost: {total_cost}, Transaction cost: {transaction_cost}")
        else:
            self.reward += -5
            self.logger.info("Not enough balance to buy.")


    def _sell(self, shares_to_sell):
        if shares_to_sell > self.shares_held:
            shares_to_sell = self.shares_held  # Can't sell more than held

        if shares_to_sell > 0:
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost

            self.balance += total_sale
            self.shares_held -= shares_to_sell
            self.total_shares_sold += shares_to_sell
            self.total_sales_value += total_sale

            self.logger.info(f"Sold {shares_to_sell} shares at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")
        else:
            self.reward += -5
            self.logger.info("No shares to sell.")


    def set_seed(self):
        # self.np_random, seed = seeding.np_random(seed)
        np.random.seed(self.seed)
        random.seed(self.seed)
        return [self.seed]


    def render(self, mode='human', close=False):
        profit = self.net_worth - self.initial_balance
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance:.2f}')
        print(f'Shares held: {self.shares_held}')
        print(f'Net worth: {self.net_worth:.2f}')
        print(f'Profit: {profit:.2f}')

In [18]:
train_env = TradingEnv(train_df)
# env = SubprocVecEnv([make_env for _ in range(4)])

{0: 0, 1: 4387, 2: 8775, 3: 11706, 4: 16019, 5: 20386, 6: 24773, 7: 29152, 8: 32109, 9: 36418, 10: 40625, 11: 44207, 12: 47796, 13: 49643, 14: 53977, 15: 58319, 16: 62703, 17: 67087, 18: 70207, 19: 74595, 20: 78966}


In [19]:
test_env = TradingEnv(test_df)

{0: 0, 21: 117, 22: 4210, 23: 8429, 24: 12817, 25: 16419}


In [20]:
# check_env(train_env)

# Train Model

## Settings

In [21]:
model = SAC(
    policy='MlpPolicy',  # Use a Multi-Layer Perceptron policy
    env=train_env,
    verbose=1,
    learning_rate=1e-4,  # Adjust learning rate if needed
    batch_size=128,      # Adjust batch size if needed
    tensorboard_log="./sac_tensorboard/",  # Directory for TensorBoard logs
    seed= seed,
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [22]:
# Set up the evaluation callback
from stable_baselines3.common.callbacks import EvalCallback
eval_callback = EvalCallback(
    test_env,                         # Evaluation environment
    best_model_save_path= Path["model"](model_num, data_num),   # Directory to save the best model
    log_path= Path["train_log"](model_num, data_num),               # Directory to save evaluation logs
    eval_freq=5000,                   # Evaluate every 5000 steps
    n_eval_episodes=3,                # Number of episodes to evaluate
    deterministic=True,               # Use deterministic actions during evaluation
    render=False                      # Disable rendering during evaluation
)

## Train

In [23]:
model.learn(
    total_timesteps=nb_steps,
    log_interval=1000,        # Log every 1000 steps
    # callback=eval_callback    # Pass the evaluation callback here
    progress_bar= True,
)

Logging to ./sac_tensorboard/SAC_43


Output()

In [24]:
# Save the trained model
model.save(Path["model"](model_num, data_num))

In [25]:
train_hist = train_env.hist
print(len(train_hist["action"]))
train_hist_df = pd.DataFrame(train_hist)

80000


In [26]:
unique_values = train_hist_df['token'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20], dtype=int64)

# Visualisaton

## Functions

In [27]:
def plot_close_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id]
    
    # Построение графика Close к индексу DataFrame
    fig = px.line(asset_data, x=asset_data.index, y='Close_orig', 
                  title=f'Close Price for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Close': 'Close Price'})
    
    # Показать график
    fig.show()

In [28]:
def plot_price_change_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id].copy()
    
    # Вычисление процентного изменения цены (Close)
    asset_data['Price_Change_Percent'] = asset_data['Close_orig'].pct_change() * 100
    
    # Построение графика изменения цены в процентах
    fig = px.line(asset_data, x=asset_data.index, y='Price_Change_Percent', 
                  title=f'Price Change Percentage for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Price_Change_Percent': 'Price Change (%)'})
    
    # Показать график
    fig.show()

In [29]:
def plot_token_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['net_worth'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['net_worth'], mode='lines', name='Net Worth'))

    # Горизонтальная линия для net_worth = 1000
    fig.add_hline(y=1000, line_color="green", name='Net Worth = 1000')

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Net Worth = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Net Worth and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Net Worth')

    # Показать график
    fig.show()

In [30]:
def plot_action_counts(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token]
    
    # Подсчет количества каждого уникального действия для данного токена
    action_counts = token_data['action'].value_counts().reset_index()
    action_counts.columns = ['action', 'count']

    # Построение бар-чарта для отображения количества каждого действия
    fig = px.bar(action_counts, x='action', y='count', title=f'Count of Actions for {token}', labels={'action': 'Action', 'count': 'Count'})

    # Показать график
    fig.show()

In [31]:
def plot_relative_change_by_token(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token].copy()

    # Вычисление относительного изменения для current_price и net_worth
    token_data['Price_Change_Percent'] = token_data['current_price'].pct_change() * 100
    token_data['NetWorth_Change_Percent'] = token_data['net_worth'].pct_change() * 100

    # Создание графика
    fig = go.Figure()

    # Линия для изменения current_price
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['Price_Change_Percent'],
                             mode='lines', name='Current Price Change (%)'))

    # Линия для изменения net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['NetWorth_Change_Percent'],
                             mode='lines', name='Net Worth Change (%)'))

    # Настройка заголовков и осей
    fig.update_layout(title=f'Relative Change of Current Price and Net Worth for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Change (%)')

    # Показать график
    fig.show()

## Plots

In [32]:
token = 19
plot_token_data(df = train_hist_df, token = token)
plot_close_by_asset(df= train_df, asset_id= token)
plot_price_change_by_asset(df= train_df, asset_id= token)
plot_relative_change_by_token(df = train_hist_df, token = token)

In [33]:
train_hist_df

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
0,49,452.087899,999.452635,9568979,0,0.000057,0.109473,0.547912,0,0.000000
1,50,520.493723,1012.168317,8399238,0,0.000059,1.272265,-0.122243,1169741,68.405824
2,51,147.197833,1009.897166,14794542,0,0.000058,0.448769,0.717196,1169741,68.405824
3,52,89.093562,992.174437,15810793,0,0.000057,3.509809,0.394736,1169741,68.405824
4,53,812.739770,980.841024,2978037,0,0.000056,2.284561,-0.811645,14002497,792.052032
...,...,...,...,...,...,...,...,...,...,...
79995,29407,944.517414,947.957685,5,7,0.688054,0.030169,-0.843956,29730,20150.725014
79996,29408,203.043908,947.146712,1104,7,0.674006,0.171099,0.785029,29730,20150.725014
79997,29409,837.048300,946.980459,163,7,0.674430,0.035106,-0.853197,30671,20784.729406
79998,29410,931.007406,946.458996,23,7,0.671808,0.110132,-0.859938,30811,20878.688512


In [34]:
train_hist_df.describe()

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
count,80000.000000,8.000000e+04,80000.000000,8.000000e+04,80000.000000,8.000000e+04,80000.000000,80000.000000,8.000000e+04,80000.000000
mean,34692.497763,5.271826e+02,1266.190877,1.535751e+08,8.322337,8.118584e+00,-0.706796,0.224087,2.622811e+10,125041.545652
std,24586.244304,8.982919e+02,889.282964,6.213683e+08,6.306412,1.889912e+01,3.423985,0.689805,8.481383e+10,132912.870807
min,49.000000,1.174266e-08,465.024852,0.000000e+00,0.000000,5.100000e-08,-50.204405,-0.999978,0.000000e+00,0.000000
25%,12843.750000,5.261818e-01,790.005710,4.800000e+01,3.000000,9.698075e-05,-3.937621,-0.428460,9.966000e+03,29034.523786
50%,30694.500000,1.409679e+02,976.784624,4.329500e+03,7.000000,5.776666e-02,0.065854,0.412654,1.851850e+06,67459.179560
75%,56115.250000,7.173786e+02,1283.348843,4.428595e+06,14.000000,2.758496e+00,0.766564,0.894404,1.654650e+09,177938.643153
max,80929.000000,5.445408e+03,9020.948068,1.105172e+10,20.000000,1.065761e+02,128.482141,0.999949,5.827758e+11,750202.159966


# Test Model

## Test

In [35]:
obs, info = test_env.reset(reset_hist=True)  # Reset hist at the beginning
for _ in range(len(test_df)):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = test_env.step(action)

    if terminated or truncated:
        obs, info = test_env.reset(reset_hist=False)  # Do not reset hist

In [36]:
test_hist = test_env.hist
test_hist_df = pd.DataFrame(test_hist)
print(len(test_hist["action"]))

20791


In [37]:
test_hist_df

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
0,49,6.317422e+01,999.064110,114,20,8.209560,0.187178,0.942475,0,0.000000
1,50,5.490129e+00,1001.604812,121,20,8.232353,0.254308,0.938482,0,0.000000
2,51,5.490129e+00,1004.409359,121,20,8.255531,-4.719995,0.941265,0,0.000000
3,52,5.490129e+00,1002.298809,121,20,8.238088,-4.579743,0.937317,0,0.000000
4,53,5.490129e+00,1009.301049,121,20,8.295958,-4.301382,0.940934,0,0.000000
...,...,...,...,...,...,...,...,...,...,...
20786,2814,2.611473e-07,671.669583,48214025,21,0.000014,-4.199256,0.951100,1351062009,32855.049347
20787,2815,2.611473e-07,670.174948,48214025,21,0.000014,-4.554949,0.958891,1351062009,32855.049347
20788,2816,2.611473e-07,663.810696,48214025,21,0.000014,-3.100719,0.975909,1351062009,32855.049347
20789,2817,2.611473e-07,667.137464,48214025,21,0.000014,-4.498838,0.957270,1351062009,32855.049347


In [38]:
unique_values = test_hist_df['token'].unique()
unique_values

array([20, 21, 22, 23, 24, 25], dtype=int64)

## Plots

In [51]:
token = 22

In [52]:
plot_token_data(df = test_hist_df, token = token)

In [53]:
plot_close_by_asset(df= test_df, asset_id= token)

## Metrics

In [42]:
def evaluate_model(hist_df, test_df, initial_balance):
    """
    Evaluate the model's performance.

    Parameters:
    - full_hist: Dictionary containing the testing history.
    - test_df: DataFrame containing the test data.
    - initial_balance: Initial balance used in the environment.

    Returns:
    - report_df: DataFrame containing performance metrics per asset.
    - overall_metrics: Dictionary containing overall performance metrics.
    """

    # Ensure timestamps are in order
    hist_df = hist_df.sort_values('current_step').reset_index(drop=True)

    # List of assets
    assets = hist_df['token'].unique()

    # Initialize report DataFrame
    report = []

    for asset_id in assets:
        asset_hist = hist_df[hist_df['token'] == asset_id]
        asset_data = test_df[test_df['Asset_ID_encoded'] == asset_id]

        # Calculate total profit/loss
        final_net_worth = asset_hist['net_worth'].iloc[-1]
        total_profit = final_net_worth - initial_balance

        # Calculate ROI
        roi = (final_net_worth - initial_balance) / initial_balance * 100

        # Calculate Sharpe Ratio
        returns = asset_hist['net_worth'].pct_change().dropna()
        if returns.std() != 0:
            sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(252)  # Assuming daily data
        else:
            sharpe_ratio = np.nan  # Undefined if no variance

        # Calculate Maximum Drawdown
        cumulative_returns = (1 + returns).cumprod()
        cumulative_max = cumulative_returns.cummax()
        drawdown = (cumulative_returns - cumulative_max) / cumulative_max
        max_drawdown = drawdown.min()

        # Calculate Win Rate
        trades = asset_hist[asset_hist['action'] != 0]
        wins = trades[trades['net_worth'].diff() > 0]
        win_rate = len(wins) / len(trades) * 100 if len(trades) > 0 else np.nan

        # Buy-and-Hold Strategy
        initial_price = asset_data['Close_orig'].iloc[0]
        final_price = asset_data['Close_orig'].iloc[-1]
        buy_and_hold_profit = (final_price - initial_price) * (initial_balance / initial_price)
        buy_and_hold_roi = (final_price - initial_price) / initial_price * 100

        # Ideal Strategy
        min_price = asset_data['Close_orig'].min()
        max_price = asset_data['Close_orig'].max()
        ideal_profit = (max_price - min_price) * (initial_balance / min_price)
        ideal_roi = (max_price - min_price) / min_price * 100

        # Collect metrics
        report.append({
            'Asset_ID': asset_id,
            'Total Profit': total_profit,
            'ROI (%)': roi,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown (%)': max_drawdown * 100,
            'Win Rate (%)': win_rate,
            'Buy-and-Hold Profit': buy_and_hold_profit,
            'Buy-and-Hold ROI (%)': buy_and_hold_roi,
            'Ideal Profit': ideal_profit,
            'Ideal ROI (%)': ideal_roi,
            'Asset Price Change (%)': (final_price - initial_price) / initial_price * 100,
        })

    # Create DataFrame from report
    report_df = pd.DataFrame(report)

    # Calculate overall metrics
    overall_profit = report_df['Total Profit'].sum()
    overall_roi = (overall_profit / (initial_balance * len(assets))) * 100
    overall_sharpe = report_df['Sharpe Ratio'].mean()
    overall_win_rate = report_df['Win Rate (%)'].mean()
    overall_buy_and_hold_profit = report_df['Buy-and-Hold Profit'].sum()
    overall_buy_and_hold_roi = (overall_buy_and_hold_profit / (initial_balance * len(assets))) * 100

    overall_metrics = {
        'Total Profit': overall_profit,
        'ROI (%)': overall_roi,
        'Sharpe Ratio': overall_sharpe,
        'Win Rate (%)': overall_win_rate,
        'Buy-and-Hold Profit': overall_buy_and_hold_profit,
        'Buy-and-Hold ROI (%)': overall_buy_and_hold_roi,
    }

    return report_df, overall_metrics

In [43]:
# Example for one model
report_df, overall_metrics = evaluate_model(test_hist_df, test_df, initial_balance = 1000)

In [44]:
report_df

,Asset_ID,Total Profit,ROI (%),Sharpe Ratio,Max Drawdown (%),Win Rate (%),Buy-and-Hold Profit,Buy-and-Hold ROI (%),Ideal Profit,Ideal ROI (%),Asset Price Change (%)
0,20,25.626740,2.562674,0.534441,-6.220686,24.637681,-118.979657,-11.897966,230.293410,23.029341,-11.897966
1,21,-144.513370,-14.451337,0.019965,-58.269871,28.919080,-279.352882,-27.935288,1596.459700,159.645970,-27.935288
2,22,55796.046571,5579.604657,0.730618,-73.894523,50.179813,253802.131121,25380.213112,293446.958188,29344.695819,25380.213112
3,23,1580.367011,158.036701,0.341684,-79.099731,49.746544,-152.651940,-15.265194,9635.856272,963.585627,-15.265194
4,24,645.822964,64.582296,0.280374,-60.514462,50.928531,594.470161,59.447016,2103.489180,210.348918,59.447016
5,25,-502.955087,-50.295509,-0.533287,-50.021776,49.047619,233.832793,23.383279,2790.670479,279.067048,23.383279


In [45]:
test_hist_df.describe()

,current_step,balance,net_worth,shares_held,token,current_price,reward,action,total_shares_sold,total_sales_value
count,20791.00000,2.079100e+04,20791.000000,2.079100e+04,20791.000000,20791.000000,20791.000000,20791.000000,2.079100e+04,20791.000000
mean,8197.78914,6.946479e+02,2536.873663,1.430050e+07,22.487903,36.156007,-2.256595,0.728469,3.261386e+08,52783.692894
std,5579.41063,5.064974e+03,5692.965664,2.121387e+07,1.349589,111.647818,3.371622,0.429897,5.453339e+08,93597.701240
min,49.00000,2.611473e-07,497.044913,1.000000e+00,20.000000,0.000011,-53.313119,-0.999989,0.000000e+00,0.000000
25%,2695.50000,9.611532e-05,864.043153,6.430000e+02,21.000000,0.000024,-4.515786,0.741094,1.690500e+04,5888.257046
50%,7818.00000,6.429754e-02,1206.835285,2.017290e+05,22.000000,0.045549,-3.584043,0.913522,2.246710e+05,29459.320886
75%,13111.50000,5.490129e+00,2205.938995,4.457905e+07,24.000000,3.247394,-0.341316,0.956306,5.158586e+08,43045.505032
max,18357.00000,5.679529e+04,56800.384390,4.823508e+07,25.000000,588.891708,53.009665,0.999347,1.859829e+09,836760.023919


In [46]:
report_df.to_csv(Path["reports"](model_num, data_num))

In [47]:
overall_metrics

{'Total Profit': 57400.39482761702,
 'ROI (%)': 956.6732471269504,
 'Sharpe Ratio': 0.22896589811450407,
 'Win Rate (%)': 42.24321142253757,
 'Buy-and-Hold Profit': 254079.44959758603,
 'Buy-and-Hold ROI (%)': 4234.657493293101}